# Overview of exercises:

## Work with "Cristiano_test_data_2" provided by Simon from the XNAT_CRUK account.  In this data set the subject position is meant to be the same for all scans.  What varies is:

### - Image contrast
### - Image FOV (resolution)
### - Different number of slices

#### Note:  All Dixon scans have the same FOV and number of planes.  Only the contrast varies.

## Exercise 1 - Copy ROI Collection to scan with different resolution but same number of planes:

### Copy ROI Collection from scan 3 ("t2_blade_tra", res = 320 x 320, 128 slices) to scan 4 ("t2_tra", res = 320 x 190, 128 slices). 

## Exercise 2 - Copy ROI Collection to scan with different resolution and different number of planes:

### Copy ROI Collection from scan 4 ("t2_tra", res = 320 x 190, 128 slices) to scan 7 ("Dixon_T1_opp", res = 384 x 240, 192 slices) 

In [60]:
import xnat
#xnat.__version__ # 0.3.21
print(f'XNATpy version = {xnat.__version__}') # 0.3.21
#import SimpleITK as sitk
import pydicom
#import dicom2nifti
import numpy as np
#import math
import matplotlib.pyplot as plt
#from matplotlib import pyplot
#import matplotlib.image as mpimg
import os, sys, time, datetime
#import natsort
#import dicom_functions
#import explore_dicom_data
#from myshow import myshow
import importlib # usage:  importlib.reload(function)
import copy
from DicomHelperFuncs import GetDicomFpaths

import CopyROIs
importlib.reload(CopyROIs)
from CopyROIs import CopyROIs

XNATpy version = 0.3.21


### Establish connection to XNAT CRUK:

In [3]:

xnatAddress = 'https://xnatcruk.icr.ac.uk/XNAT_CRUK_ACTIVE'
#xnatAddress = 'https://xnatcruk.icr.ac.uk'
#xnatAddress = 'http://xnatcruk.icr.ac.uk/XNAT_CRUK_ACTIVE'

session = xnat.connect(xnatAddress, user='ctorti', password='ctorti@gmail.com', debug=False)
#session = xnat.connect(xnatAddress, user='ctorti', password='ctorti@gmail.com', debug=True)

AttributeError: 'NoneType' object has no attribute 'startswith'

### March 3:

### Since I was unable to establish a connection I downloaded the entire patient collection using the web interface.

### The root folder of the files are at:

### C:\Data\Cristiano_test_data_2\20170302_112746_Aera

### In my own XNAT instance I used the XNAT Compressed Uploader (Option 2 using the web interface) I selected the zip file that was downloaded in the XNAT_CRUK session.

### I loaded the images in the OHIF viewer and selected Dixon_T1_opp (scan session #7), and drew some contours in the 11th frame.

### March 4:

### I later drew contours on slice 5 of scan 3 ("t2_blade_tra") and named it "ROIs_t2_blade_tra_slice_5". 

# Connect to my XNAT instance:

In [4]:
xnatAddress = 'http://10.1.1.17'

session = xnat.connect(xnatAddress, user='admin', password='admin')
session

In [126]:
# Create a dictionary to store various variables to input into CopyROI():

todaysDate = time.strftime("%Y-%m-%d", time.gmtime())

rootDir = os.path.join(r'C:\Temp', todaysDate)

xnatDict = {# Define the XNAT login details:
            'xnatAddress':'http://10.1.1.17',\
            'username':'admin',\
            'password':'admin',\
    
            # Define the REST variables that point to the ROIs to be copied:
            'fromRoiProjLab':'TestData',\
            'fromRoiSubjLab':'Volunteer_1',\
            'fromRoiExpLab':'20170302_112746_Aera',\
            'fromRoiLab':'AIM_20200304_110009',\
    
            # Define the REST variables that point to the DICOM files that the ROI applies to:
            'fromDicomProjLab':'TestData',\
            'fromDicomSubjLab':'Volunteer_1',\
            'fromDicomExpLab':'20170302_112746_Aera',\
            'fromDicomScanLab':'3',\
    
            # Define the REST variables that point to the DICOM files that the ROI is to be applied to:
            'toDicomProjLab':'TestData',\
            'toDicomSubjLab':'Volunteer_1',\
            'toDicomExpLab':'20170302_112746_Aera',\
            'toDicomScanLab':'4',\
    
            # Define the label to assign to Structure Set Label and to ROI Name of the new RTSTUCT file:
            #'toRoiStructSetLab' = 'ROIs_t2_blade_tra_slice_5'
    
    
            # Define the directories to download files to:
            'rootDir':rootDir, \
            'fromDicomDir':os.path.join(rootDir, 'From DICOM'),\
            'fromRoiDir':os.path.join(rootDir, 'From RTSTRUCT'),\
            'toDicomDir':os.path.join(rootDir, 'To DICOM'),\
            'toRoiDir':os.path.join(rootDir, 'To RTSTRUCT')
           }

xnatDict

{'xnatAddress': 'http://10.1.1.17',
 'username': 'admin',
 'password': 'admin',
 'fromRoiProjLab': 'TestData',
 'fromRoiSubjLab': 'Volunteer_1',
 'fromRoiExpLab': '20170302_112746_Aera',
 'fromRoiLab': 'AIM_20200304_110009',
 'fromDicomProjLab': 'TestData',
 'fromDicomSubjLab': 'Volunteer_1',
 'fromDicomExpLab': '20170302_112746_Aera',
 'fromDicomScanLab': '3',
 'toDicomProjLab': 'TestData',
 'toDicomSubjLab': 'Volunteer_1',
 'toDicomExpLab': '20170302_112746_Aera',
 'toDicomScanLab': '4',
 'rootDir': 'C:\\Temp\\2020-03-04',
 'fromDicomDir': 'C:\\Temp\\2020-03-04\\From DICOM',
 'fromRoiDir': 'C:\\Temp\\2020-03-04\\From RTSTRUCT',
 'toDicomDir': 'C:\\Temp\\2020-03-04\\To DICOM',
 'toRoiDir': 'C:\\Temp\\2020-03-04\\To RTSTRUCT'}

RoiProj = session.projects[xnatDict['RoiProjLab']]
RoiSubj = RoiProj.subjects[xnatDict['RoiSubjLab']]
RoiExp = RoiSubj.experiments[xnatDict['RoiExpLab']]

roi = RoiExp.assessors[xnatDict['RoiLab']].resources['RTSTRUCT'].files[0]

roiId = roi.id
roiId

session.projects[xnatDict['fromDicomProjLab']]
dicomsProj = session.projects[xnatDict['fromDicomProjLab']]
dicomsProj.subjects[xnatDict['fromDicomSubjLab']] 
dicomsSubj = dicomsProj.subjects[xnatDict['fromDicomSubjLab']] 
#dicomsSubj.experiments
dicomsSubj.experiments[xnatDict['fromDicomExpLab']]
dicomsExp = dicomsSubj.experiments[xnatDict['fromDicomExpLab']]
#dicomsExp.scans
dicomsExp.scans[xnatDict['fromDicomScanLab']]
dicoms = dicomsExp.scans[xnatDict['fromDicomScanLab']]
dicoms

# Exercise 1 - Copy ROI Collection to scan with different resolution but same number of planes:

### Copy ROI Collection from scan 3 ("t2_blade_tra", res = 320 x 320, 128 slices) to scan 4 ("t2_tra", res = 320 x 190, 128 slices). 

In [93]:
import CopyROIs
importlib.reload(CopyROIs)
from CopyROIs import CopyROIs

import ModifyROIs
importlib.reload(ModifyROIs)
from ModifyROIs import ModifyROIs


# Use CopyROIs to copy the data locally:
xnatDict = CopyROIs(xnatDict, del_dicoms=False, debug=False)

xnatDict

100% of  12.2 KiB |################################|  11.9 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  53.7 MiB/s Time:  0:00:00


100% of 374.6 KiB |################################|  37.2 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  41.8 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  21.9 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  27.0 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  26.1 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  11.5 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  54.7 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  53.7 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  46.8 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  53.7 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  51.9 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  41.8 MiB/s Time:  0:00:00
100% of 374.6 KiB |#####################

100% of 374.6 KiB |################################|  27.3 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  12.4 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  22.2 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  24.0 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  24.4 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  31.7 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  26.3 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  32.9 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  23.6 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  44.6 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  32.7 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  41.8 MiB/s Time:  0:00:00
100% of 374.6 KiB |#####################

100% of 293.7 KiB |################################|  40.9 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  56.2 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  55.1 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  36.8 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  40.5 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  45.1 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  40.0 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  44.3 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  36.8 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  41.7 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  58.9 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  41.2 MiB/s Time:  0:00:00
100% of 293.7 KiB |#####################

100% of 293.7 KiB |################################|  40.1 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  40.1 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  48.2 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  32.4 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  24.2 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  21.8 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  24.7 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  14.2 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  29.3 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  29.5 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  34.5 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  38.0 MiB/s Time:  0:00:00
100% of 293.7 KiB |#####################


Reading the SOP Instance UIDs from the DICOMs in:
 C:\Temp\2020-03-04\From DICOM

Looping through each of 1 Contour Image Sequences to read the Referenced SOP Instance UIDs and find the indeces of the matching SOP Instance UIDs in the DICOMs..

The matching indeces are:
 [4]

Looping through each of 3 Contour Sequences to read the Referenced SOP Instance UIDs and find the indeces of the matching SOP Instance UIDs in the DICOMs..

The matching indeces are:
 [4, 4, 4]

The Study Date are the same:
 20170302

Changing the Study Time from:
 114247 
to:
 112746

The Patient Name are the same:
 Volunteer_1

The Patient ID are the same:
 20170302_112746_Aera

Patient Birth Date does not exist in the DICOMs

The Patient Sex are the same:
 F

The Study Instance UID are the same:
 1.3.12.2.1107.5.2.18.41171.30000017030208045014800000004

The Study Instance UID are the same:
 1.3.12.2.1107.5.2.18.41171.30000017030208045014800000004

Changing the Series Instance UID from:
 1.3.12.2.1107.5.2.18.41

{'xnatAddress': 'http://10.1.1.17',
 'username': 'admin',
 'password': 'admin',
 'roiProjLab': 'TestData',
 'roiSubjLab': 'Volunteer_1',
 'roiExpLab': '20170302_112746_Aera',
 'roiLab': 'AIM_20200304_110009',
 'fromDicomProjLab': 'TestData',
 'fromDicomSubjLab': 'Volunteer_1',
 'fromDicomExpLab': '20170302_112746_Aera',
 'fromDicomScanLab': '3',
 'toDicomProjLab': 'TestData',
 'toDicomSubjLab': 'Volunteer_1',
 'toDicomExpLab': '20170302_112746_Aera',
 'toDicomScanLab': '4',
 'rootDir': 'C:\\Temp\\2020-03-04',
 'fromDicomDir': 'C:\\Temp\\2020-03-04\\From DICOM',
 'fromRoiDir': 'C:\\Temp\\2020-03-04\\From RTSTRUCT',
 'toDicomDir': 'C:\\Temp\\2020-03-04\\To DICOM',
 'toRoiDir': 'C:\\Temp\\2020-03-04\\To RTSTRUCT',
 'toRoiFname': 'AIM_20200304_152939.dcm',
 'toRoiFpath': 'C:\\Temp\\2020-03-04\\To RTSTRUCT\\AIM_20200304_152939.dcm',
 'jsonFname': 'AIM_20200304_152939.json',
 'jsonFpath': 'C:\\Temp\\2020-03-04\\To RTSTRUCT\\AIM_20200304_152939.json'}

In [100]:
# Test stuff:

dicomFpaths1 = GetDicomFpaths(xnatDict['fromDicomDir'])
dicomFpaths2 = GetDicomFpaths(xnatDict['toDicomDir'])

# The ROI:
#roi1 = pydicom.dcmread(r'C:\Temp\2020-03-04\From RTSTRUCT\AIM_20200304_110009.dcm')
#roi1

# load a dicom from the "from" series:
#dicom1 = pydicom.dcmread(r'C:\Temp\2020-03-04\From DICOM\1.3.12.2.1107.5.2.18.41171.30000017030208045014800000004-3-1-1y5alqg.dcm')
dicom1 = pydicom.dcmread(dicomFpaths1[0])
dicom1


(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'M', 'NORM', 'DIS3D', 'DIS2D']
(0008, 0012) Instance Creation Date              DA: '20170302'
(0008, 0013) Instance Creation Time              TM: '114247.966000'
(0008, 0016) SOP Class UID                       UI: MR Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.3.12.2.1107.5.2.18.41171.2017030211424716885123155
(0008, 0020) Study Date                          DA: '20170302'
(0008, 0021) Series Date                         DA: '20170302'
(0008, 0022) Acquisition Date                    DA: '20170302'
(0008, 0023) Content Date                        DA: '20170302'
(0008, 0030) Study Time                          TM: '112746.335000'
(0008, 0031) Series Time                         TM: '114247.131000'
(0008, 0032) Acquisition Time                    TM: '113703.612500'
(0008, 0033) Content Time                        TM: '

In [101]:
# load a dicom from the "to" series:
#dicom2 = pydicom.dcmread(r'C:\Temp\2020-03-04\To DICOM\1.3.12.2.1107.5.2.18.41171.30000017030208045014800000004-4-1-1i5qe5f.dcm')
dicom2 = pydicom.dcmread(dicomFpaths2[0])
dicom2

(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'M', 'NORM', 'DIS3D', 'DIS2D']
(0008, 0012) Instance Creation Date              DA: '20170302'
(0008, 0013) Instance Creation Time              TM: '114438.029000'
(0008, 0016) SOP Class UID                       UI: MR Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.3.12.2.1107.5.2.18.41171.2017030211443692233924053
(0008, 0020) Study Date                          DA: '20170302'
(0008, 0021) Series Date                         DA: '20170302'
(0008, 0022) Acquisition Date                    DA: '20170302'
(0008, 0023) Content Date                        DA: '20170302'
(0008, 0030) Study Time                          TM: '112746.335000'
(0008, 0031) Series Time                         TM: '114436.898000'
(0008, 0032) Acquisition Time                    TM: '114259.012500'
(0008, 0033) Content Time                        TM: '

In [82]:

print('dicom1.StudyInstanceUID =', dicom1.StudyInstanceUID)
print('dicom1.SeriesInstanceUID =', dicom1.SeriesInstanceUID)
print('dicom1.SOPInstanceUID =', dicom1.SOPInstanceUID)
#print('dicom1.ImagePositionPatient =', dicom1.ImagePositionPatient)
#print('dicom1.ImageOrientationPatient =', dicom1.ImageOrientationPatient)
#print('dicom1.SliceLocation =', dicom1.SliceLocation)
print('')
print('dicom2.StudyInstanceUID =', dicom2.StudyInstanceUID)
print('dicom2.SeriesInstanceUID =', dicom2.SeriesInstanceUID)
print('dicom2.SOPInstanceUID =', dicom2.SOPInstanceUID)
#print('dicom2.ImagePositionPatient =', dicom2.ImagePositionPatient)
#print('dicom2.ImageOrientationPatient =', dicom2.ImageOrientationPatient)
#print('dicom2.SliceLocation =', dicom2.SliceLocation)

dicom1.StudyInstanceUID = 1.3.12.2.1107.5.2.18.41171.30000017030208045014800000004
dicom1.SeriesInstanceUID = 1.3.12.2.1107.5.2.18.41171.201703021136315076922448.0.0.0
dicom1.SOPInstanceUID = 1.3.12.2.1107.5.2.18.41171.2017030211424716885123155

dicom2.StudyInstanceUID = 1.3.12.2.1107.5.2.18.41171.30000017030208045014800000004
dicom2.SeriesInstanceUID = 1.3.12.2.1107.5.2.18.41171.2017030211424753515923220.0.0.0
dicom2.SOPInstanceUID = 1.3.12.2.1107.5.2.18.41171.2017030211443692233924053


In [84]:
dicom2\
.StudyInstanceUID

'1.3.12.2.1107.5.2.18.41171.30000017030208045014800000004'

In [85]:
dicom2.\
StudyInstanceUID

'1.3.12.2.1107.5.2.18.41171.30000017030208045014800000004'

### I uploaded AIM_20200304_152939.dcm using the ROIUploadAssistant tool to XNAT and can see it under the list of Assessments.

### The contours are roughly where they ought to be but there appears to be a shift in Y.

### Will see if I can figure out why...

In [104]:
roiFname1 = r'AIM_20200304_110009.dcm'
roiFname2 = r'AIM_20200304_152939.dcm'

roiFpath1 = os.path.join(xnatDict['fromRoiDir'], roiFname1)
roiFpath2 = os.path.join(xnatDict['toRoiDir'], roiFname2)

roi1 = pydicom.dcmread(roiFpath1)
roi2 = pydicom.dcmread(roiFpath2)

In [105]:
if 0:
    print('dicom1.FrameOfReferenceUID =', dicom1.FrameOfReferenceUID)
    print('dicom2.FrameOfReferenceUID =', dicom2.FrameOfReferenceUID)
    print('')
    print('dicom1.InStackPositionNumber =', dicom1.InStackPositionNumber)
    print('dicom2.InStackPositionNumber =', dicom2.InStackPositionNumber)
    print('')
    print('dicom1.ImagesInAcquisition =', dicom1.ImagesInAcquisition)
    print('dicom2.ImagesInAcquisition =', dicom2.ImagesInAcquisition)
    print('')
print('dicom1.ImagePositionPatient =', dicom1.ImagePositionPatient)
print('dicom2.ImagePositionPatient =', dicom2.ImagePositionPatient)
print('')
print('dicom1.ImageOrientationPatient =', dicom1.ImageOrientationPatient)
print('dicom2.ImageOrientationPatient =', dicom2.ImageOrientationPatient)
print('')
print('dicom1.SliceLocation =', dicom1.SliceLocation)
print('dicom2.SliceLocation =', dicom2.SliceLocation)
print('')







dicom1.ImagePositionPatient = ['-255.81113815308', '-279.05569076538', '142.46780395508']
dicom2.ImagePositionPatient = ['-255.81113815308', '-168.55297851563', '149.91796875']

dicom1.ImageOrientationPatient = ['1', '0', '0', '0', '1', '0']
dicom2.ImageOrientationPatient = ['1', '0', '0', '0', '1', '0']

dicom1.SliceLocation = 142.46780395508
dicom2.SliceLocation = 149.91796875



In [109]:
imPositions1 = []
imPositions2 = []

print('\nImage positions for dicom1:')
for f in range(len(dicomFpaths1)):
    dicom = pydicom.dcmread(dicomFpaths1[f])
    imPositions1.append(dicom.ImagePositionPatient)
    
    print(dicom.ImagePositionPatient)
    

# It's the 5th slice in the first set that has the contours:
print('\nImage position of 5th slice in dicom1:\n', imPositions1[4])


print('\nImage positions for dicom2:')
for f in range(len(dicomFpaths2)):
    dicom = pydicom.dcmread(dicomFpaths2[f])
    imPositions2.append(dicom.ImagePositionPatient)
    
    print(dicom.ImagePositionPatient)


Image positions for dicom1:
['-255.81113815308', '-279.05569076538', '142.46780395508']
['-255.81113815308', '-279.05569076538', '140.0677947998']
['-255.81113815308', '-279.05569076538', '137.66780090332']
['-255.81113815308', '-279.05569076538', '135.26779174805']
['-255.81113815308', '-279.05569076538', '132.86779785156']
['-255.81113815308', '-279.05569076538', '130.46780395508']
['-255.81113815308', '-279.05569076538', '128.0677947998']
['-255.81113815308', '-279.05569076538', '125.66780090332']
['-255.81113815308', '-279.05569076538', '123.26779174805']
['-255.81113815308', '-279.05569076538', '120.86779785156']
['-255.81113815308', '-279.05569076538', '118.46780395508']
['-255.81113815308', '-279.05569076538', '116.0677947998']
['-255.81113815308', '-279.05569076538', '113.66779327393']
['-255.81113815308', '-279.05569076538', '111.26779937744']
['-255.81113815308', '-279.05569076538', '108.86779785156']
['-255.81113815308', '-279.05569076538', '106.46779632568']
['-255.8111381

['-255.81113815308', '-168.55297851563', '104.31797027588']
['-255.81113815308', '-168.55297851563', '101.91797637939']
['-255.81113815308', '-168.55297851563', '99.517974853516']
['-255.81113815308', '-168.55297851563', '97.117973327637']
['-255.81113815308', '-168.55297851563', '94.717971801758']
['-255.81113815308', '-168.55297851563', '92.317970275879']
['-255.81113815308', '-168.55297851563', '89.917976379395']
['-255.81113815308', '-168.55297851563', '87.517974853516']
['-255.81113815308', '-168.55297851563', '85.117973327637']
['-255.81113815308', '-168.55297851563', '82.717971801758']
['-255.81113815308', '-168.55297851563', '80.317970275879']
['-255.81113815308', '-168.55297851563', '77.917976379395']
['-255.81113815308', '-168.55297851563', '75.517974853516']
['-255.81113815308', '-168.55297851563', '73.117973327637']
['-255.81113815308', '-168.55297851563', '70.717971801758']
['-255.81113815308', '-168.55297851563', '68.317970275879']
['-255.81113815308', '-168.55297851563',

In [110]:
roi1

(0008, 0016) SOP Class UID                       UI: RT Structure Set Storage
(0008, 0018) SOP Instance UID                    UI: 1.2.826.0.1.534147.578.2126823365.20202413735791
(0008, 0020) Study Date                          DA: '20170302'
(0008, 0030) Study Time                          TM: '114247'
(0008, 0050) Accession Number                    SH: ''
(0008, 0060) Modality                            CS: 'RTSTRUCT'
(0008, 0070) Manufacturer                        LO: 'SIEMENS'
(0008, 0090) Referring Physician's Name          PN: ''
(0008, 1070) Operators' Name                     PN: ''
(0008, 1090) Manufacturer's Model Name           LO: 'Aera'
(0010, 0010) Patient's Name                      PN: 'Volunteer_1'
(0010, 0020) Patient ID                          LO: '20170302_112746_Aera'
(0010, 0030) Patient's Birth Date                DA: '00021130'
(0010, 0040) Patient's Sex                       CS: 'F'
(0018, 1020) Software Version(s)                 LO: 'syngo MR E11'
(0020, 

# Need to re-think the approach I've been taking...

In [132]:
import DicomHelperFuncs
importlib.reload(DicomHelperFuncs)
from DicomHelperFuncs import GetDicomData

# Get the data from of the DICOM files that have contours:
dicomDict1 = GetDicomData(xnatDict['fromDicomDir'])

dicomDict1

{'1.3.12.2.1107.5.2.18.41171.2017030211424716885123155': {'Dicom frame no': 1,
  'Dicom fpath': 'C:\\Temp\\2020-03-04\\From DICOM\\1.3.12.2.1107.5.2.18.41171.30000017030208045014800000004-3-1-1y5alqg.dcm',
  'Image pos': [-255.81113815308, -279.05569076538, 142.46780395508],
  'Image orient': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
  'Slice loc': 142.46780395508,
  'Pix spacing': [1.5625, 1.5625],
  'Pix array': array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]], dtype=uint16)},
 '1.3.12.2.1107.5.2.18.41171.2017030211424737916723219': {'Dicom frame no': 2,
  'Dicom fpath': 'C:\\Temp\\2020-03-04\\From DICOM\\1.3.12.2.1107.5.2.18.41171.30000017030208045014800000004-3-2-6f3ot1.dcm',
  'Image pos': [-255.81113815308, -279.05569076538, 140.0677947998],
  'Image orient': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
  'Slice loc': 140.0677947998,
  'P

In [174]:
import DicomHelperFuncs
importlib.reload(DicomHelperFuncs)
from DicomHelperFuncs import GetRoiData

# The ROI fname:
roiFname = xnatDict['fromRoiLab'] + '.dcm'

# The ROI fpath:
roiFpath = os.path.join(xnatDict['fromRoiDir'], roiFname)

""" Consider adding roiFpath to xnatDict..."""

# Get the contour data from of the DICOM-RTSTRUCT file:
roiDict1 = GetRoiData(roiFpath)

roiDict1

{'1.3.12.2.1107.5.2.18.41171.2017030211424716719523153': {'Roi fpath': 'C:\\Temp\\2020-03-04\\From RTSTRUCT\\AIM_20200304_110009.dcm',
  'No of contours': 3,
  'Contour nos': [1, 2, 3],
  'No of contour pts': [44, 45, 116],
  'Contour data': [[-18.083705824172,
    -62.778718089179,
    132.86779785156,
    -18.594431053999,
    -61.2465423997,
    132.86779785156,
    -19.105156283825,
    -59.714366710221,
    132.86779785156,
    -19.105156283825,
    -57.671465790916,
    132.86779785156,
    -17.06225536452,
    -57.671465790916,
    132.86779785156,
    -15.019354445214,
    -57.671465790916,
    132.86779785156,
    -13.487178755735,
    -56.650015331263,
    132.86779785156,
    -11.955003066256,
    -56.139290101437,
    132.86779785156,
    -10.422827376777,
    -55.117839641784,
    132.86779785156,
    -9.9121021469513,
    -53.585663952305,
    132.86779785156,
    -9.401376917125,
    -52.053488262826,
    132.86779785156,
    -7.8692012276459,
    -51.542763033,
    132.

In [142]:
import DicomHelperFuncs
importlib.reload(DicomHelperFuncs)
from DicomHelperFuncs import CombineDicomAndRoiData

dicomAndRoiDict = CombineDicomAndRoiData(dicomDict, roiDict)

dicomAndRoiDict

{'1.3.12.2.1107.5.2.18.41171.2017030211424716885123155': {'Dicom frame no': 1,
  'Dicom fpath': 'C:\\Temp\\2020-03-04\\From DICOM\\1.3.12.2.1107.5.2.18.41171.30000017030208045014800000004-3-1-1y5alqg.dcm',
  'Image pos': [-255.81113815308, -279.05569076538, 142.46780395508],
  'Image orient': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
  'Slice loc': 142.46780395508,
  'Pix spacing': [1.5625, 1.5625],
  'Pix array': array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]], dtype=uint16),
  'No of contours': 0,
  'Contour nos': [],
  'No of contour pts': 0,
  'Contour data': []},
 '1.3.12.2.1107.5.2.18.41171.2017030211424737916723219': {'Dicom frame no': 2,
  'Dicom fpath': 'C:\\Temp\\2020-03-04\\From DICOM\\1.3.12.2.1107.5.2.18.41171.30000017030208045014800000004-3-2-6f3ot1.dcm',
  'Image pos': [-255.81113815308, -279.05569076538, 140.067794

In [133]:
import DicomHelperFuncs
importlib.reload(DicomHelperFuncs)
from DicomHelperFuncs import GetDicomData

# Get the data from of the DICOM files that don't have contours:
dicomDict2 = GetDicomData(xnatDict['toDicomDir'])

dicomDict2

{'1.3.12.2.1107.5.2.18.41171.2017030211443692233924053': {'Dicom frame no': 1,
  'Dicom fpath': 'C:\\Temp\\2020-03-04\\To DICOM\\1.3.12.2.1107.5.2.18.41171.30000017030208045014800000004-4-1-1i5qe5f.dcm',
  'Image pos': [-255.81113815308, -168.55297851563, 149.91796875],
  'Image orient': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
  'Slice loc': 149.91796875,
  'Pix spacing': [1.5625, 1.5625],
  'Pix array': array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]], dtype=uint16)},
 '1.3.12.2.1107.5.2.18.41171.201703021144378761324117': {'Dicom frame no': 2,
  'Dicom fpath': 'C:\\Temp\\2020-03-04\\To DICOM\\1.3.12.2.1107.5.2.18.41171.30000017030208045014800000004-4-2-oocxno.dcm',
  'Image pos': [-255.81113815308, -168.55297851563, 147.51797485352],
  'Image orient': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
  'Slice loc': 147.51797485352,
  'Pix spacin

# For each Contour Sequence in roi1, using the corresponding Image position, find the slice in dicom2 that is closest to the Image position of the slice from dicom1:

In [ ]:
import DicomHelperFuncs
importlib.reload(DicomHelperFuncs)
from DicomHelperFuncs import GetDicomData
from DicomHelperFuncs import GetRoiData

# Inputs:
# dicomAndRoiDict1 (combination of dicomDict1 and roiDict1)
# dicomDict2

# Get keys (SOP UIDs) from dicomAndRoiDict1:
keys1 = list(dicomAndRoiDict1.keys())

# and the keys from dicomDict2:
keys2 = list(dicomDict2.keys())


# Cycle through each key in dicomAndRoiDict1, if 
for key1 in keys1:
    if dicomAndRoiDict1[key]['No of contours']!=0:
        
        """ Instead of continuing I think it would make sense to create
        a new dictionary that links the positions in dicomDict1 to those
        in dicomDict2 ..."""


# Map out the connection between dicom2 and dicom1: dicomDict2_to_1:

In [165]:
import DicomHelperFuncs
importlib.reload(DicomHelperFuncs)
from DicomHelperFuncs import GetDicomData
from DicomHelperFuncs import GetRoiData

# Inputs:
# dicomDict1 
# dicomDict2

# Outputs:
# dicomDict2_to_1

# Chose whether to find the closest matching slice along x, y or z directions, or r:
searchBy = 'z'
#searchBy = 'r'


# Initialise dicomDict2_to_1 as a copy of dicomDict2:
dicomDict2_to_1 = copy.deepcopy(dicomDict2)

# For every key (SOP UID) in dicomDict2 find the SOP UID of the nearest slice in dicomDict1:
keys1 = list(dicomDict1.keys())
keys2 = list(dicomDict2.keys())

# Store the indices of the keys of dicomDict1 whose positions are closest to the positions 
# of the slices in dicomDict2 for both the z direction and overall:
indsX = []
indsY = []
indsZ = []
indsR = []

for key2 in keys2:
    pos2 = dicomDict2[key2]['Image pos']
    
    #print('\npos2 =', pos2, '\n')
    
    # Create a list of the positional differences between pos2 and every slice in dicomDict1
    # along x, y, z and the vector:
    dX = []
    dY = []
    dZ = []
    dR = []
    
    for key1 in keys1:
        pos1 = dicomDict1[key1]['Image pos']
        
        #print('pos1 =', pos1)
        
        # Calculate the distance between pos1 and pos2 along x, y and z:
        dx = pos2[0] - pos1[0]
        dy = pos2[1] - pos1[1]
        dz = pos2[2] - pos1[2]
        
        dr = (dx**2 + dy**2 + dz**2)**.5
        
        dX.append(dx)
        dY.append(dy)
        dZ.append(dz)
        dR.append(dr)
        
        #print('dx, dy, dz, dr =', dx, dy, dz, dr)
        
    # Find the index of the minimum positional difference along different directions:
    indX = dX.index(min(dX))
    indY = dY.index(min(dY))
    indZ = dZ.index(min(dZ))
    indR = dR.index(min(dR))
    
    # Store these indices:
    indsX.append(indX)
    indsY.append(indY)
    indsZ.append(indZ)
    indsR.append(indR)
    
    # Update dicomDict2_to_1 with the details of dicomDict1 that most closesly match in position 
    # based on the chosen searchBy:
    if searchBy=='x':
        ind = copy.deepcopy(indX)
    if searchBy=='y':
        ind = copy.deepcopy(indY)
    if searchBy=='z':
        ind = copy.deepcopy(indZ)
    if searchBy=='r':
        ind = copy.deepcopy(indR)
        
    dicomDict2_to_1[key2].update({'Closest Dicom frame no':dicomDict1[keys1[ind]]['Dicom frame no'], \
                                  'Closest Dicom fpath':dicomDict1[keys1[ind]]['Dicom fpath'], \
                                  'Closest Image pos':dicomDict1[keys1[ind]]['Image pos'], \
                                  'Closest Image orient':dicomDict1[keys1[ind]]['Image orient'], \
                                  'Closest Slice loc':dicomDict1[keys1[ind]]['Slice loc'], \
                                  'Closest Pix spacing':dicomDict1[keys1[ind]]['Pix spacing'], \
                                  'x2 - x1':dX[ind],
                                  'y2 - y1':dY[ind],
                                  'z2 - z1':dZ[ind],
                                  'r2 - r1':dR[ind],
                                 })
    
    #print('\n\n')
        
#inds
dicomDict2_to_1
#indsZ

{'1.3.12.2.1107.5.2.18.41171.2017030211443692233924053': {'Dicom frame no': 1,
  'Dicom fpath': 'C:\\Temp\\2020-03-04\\To DICOM\\1.3.12.2.1107.5.2.18.41171.30000017030208045014800000004-4-1-1i5qe5f.dcm',
  'Image pos': [-255.81113815308, -168.55297851563, 149.91796875],
  'Image orient': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
  'Slice loc': 149.91796875,
  'Pix spacing': [1.5625, 1.5625],
  'Pix array': array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]], dtype=uint16),
  'Closest Dicom frame no': 1,
  'Closest Dicom fpath': 'C:\\Temp\\2020-03-04\\From DICOM\\1.3.12.2.1107.5.2.18.41171.30000017030208045014800000004-3-1-1y5alqg.dcm',
  'Closest Image pos': [-255.81113815308, -279.05569076538, 142.46780395508],
  'Closest Image orient': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
  'Closest Slice loc': 142.46780395508,
  'Closest Pix spacing': [

# Instead of dicomDict2_to_1 I think dicomDict1_to_2 would be more useful:

In [168]:
import DicomHelperFuncs
importlib.reload(DicomHelperFuncs)
from DicomHelperFuncs import GetDicomData
from DicomHelperFuncs import GetRoiData

# Inputs:
# dicomDict1 
# dicomDict2
# searchBy

# Outputs:
# dicomDict1_to_2

# Chose whether to find the closest matching slice along x, y or z directions, or r:
searchBy = 'z'
#searchBy = 'r'


# Initialise dicomDict1_to_2 as a copy of dicomDict1:
dicomDict1_to_2 = copy.deepcopy(dicomDict1)

# For every key (SOP UID) in dicomDict1 find the SOP UID of the nearest slice in dicomDict2:
keys1 = list(dicomDict1.keys())
keys2 = list(dicomDict2.keys())

# Store the indices of the keys of dicomDict2 whose positions are closest to the positions 
# of the slices in dicomDict1 for both the z direction and overall:
indsX = []
indsY = []
indsZ = []
indsR = []

for key1 in keys1:
    pos1 = dicomDict1[key1]['Image pos']
    
    #print('\npos1 =', pos1, '\n')
    
    # Create a list of the positional differences between pos2 and every slice in dicomDict1
    # along x, y, z and the vector:
    dX = []
    dY = []
    dZ = []
    dR = []
    
    for key2 in keys2:
        pos2 = dicomDict2[key2]['Image pos']
        
        #print('pos2 =', pos2)
        
        # Calculate the distance between pos1 and pos2 along x, y and z:
        dx = pos2[0] - pos1[0]
        dy = pos2[1] - pos1[1]
        dz = pos2[2] - pos1[2]
        
        dr = (dx**2 + dy**2 + dz**2)**.5
        
        dX.append(dx)
        dY.append(dy)
        dZ.append(dz)
        dR.append(dr)
        
        #print('dx, dy, dz, dr =', dx, dy, dz, dr)
        
    # Find the index of the minimum positional difference along different 
    # directions:
    #indX = dX.index(min(dX))
    #indY = dY.index(min(dY))
    #indZ = dZ.index(min(dZ))
    #indR = dR.index(min(dR))
        
    # Find the index of the absolution minimum positional difference along different directions:
    indX = [abs(dx) for dx in dX].index(min([abs(dx) for dx in dX]))
    indY = [abs(dy) for dx in dY].index(min([abs(dy) for dx in dY]))
    indZ = [abs(dz) for dx in dZ].index(min([abs(dz) for dx in dZ]))
    indR = [abs(dr) for dx in dR].index(min([abs(dr) for dx in dR]))
    
    # Store these indices:
    indsX.append(indX)
    indsY.append(indY)
    indsZ.append(indZ)
    indsR.append(indR)
    
    # Update dicomDict1_to_2 with the details of dicomDict2 that most closesly match in position 
    # based on the chosen searchBy:
    if searchBy=='x':
        ind = copy.deepcopy(indX)
    if searchBy=='y':
        ind = copy.deepcopy(indY)
    if searchBy=='z':
        ind = copy.deepcopy(indZ)
    if searchBy=='r':
        ind = copy.deepcopy(indR)
        
    dicomDict1_to_2[key1].update({'Closest Dicom SOP UID':keys2[ind], \
                                  'Closest Dicom frame no':dicomDict2[keys2[ind]]['Dicom frame no'], \
                                  'Closest Dicom fpath':dicomDict2[keys2[ind]]['Dicom fpath'], \
                                  'Closest Image pos':dicomDict2[keys2[ind]]['Image pos'], \
                                  'Closest Image orient':dicomDict2[keys2[ind]]['Image orient'], \
                                  'Closest Slice loc':dicomDict2[keys2[ind]]['Slice loc'], \
                                  'Closest Pix spacing':dicomDict2[keys2[ind]]['Pix spacing'], \
                                  'x2 - x1':dX[ind],
                                  'y2 - y1':dY[ind],
                                  'z2 - z1':dZ[ind],
                                  'r2 - r1':dR[ind],
                                 })
    
    #print('\n\n')
        
#inds
dicomDict1_to_2
#indsZ

{'1.3.12.2.1107.5.2.18.41171.2017030211424716885123155': {'Dicom frame no': 1,
  'Dicom fpath': 'C:\\Temp\\2020-03-04\\From DICOM\\1.3.12.2.1107.5.2.18.41171.30000017030208045014800000004-3-1-1y5alqg.dcm',
  'Image pos': [-255.81113815308, -279.05569076538, 142.46780395508],
  'Image orient': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
  'Slice loc': 142.46780395508,
  'Pix spacing': [1.5625, 1.5625],
  'Pix array': array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]], dtype=uint16),
  'Closest Dicom SOP UID': '1.3.12.2.1107.5.2.18.41171.2017030211443692233924053',
  'Closest Dicom frame no': 1,
  'Closest Dicom fpath': 'C:\\Temp\\2020-03-04\\To DICOM\\1.3.12.2.1107.5.2.18.41171.30000017030208045014800000004-4-1-1i5qe5f.dcm',
  'Closest Image pos': [-255.81113815308, -168.55297851563, 149.91796875],
  'Closest Image orient': [1.0, 0.0, 0

# Test out above wrapped as a function:

In [169]:
import DicomHelperFuncs
importlib.reload(DicomHelperFuncs)
from DicomHelperFuncs import GetDicomData
from DicomHelperFuncs import GetRoiData
from DicomHelperFuncs import GetClosestDicomPositions

# Chose whether to find the closest matching slice along x, y or z directions, or r:
searchBy = 'z'
#searchBy = 'r'

# Use GetClosestDicomPositions:
dicomDict1_to_2 = GetClosestDicomPositions(dicomDict1, dicomDict2, searchBy)

dicomDict1_to_2

{'1.3.12.2.1107.5.2.18.41171.2017030211424716885123155': {'Dicom frame no': 1,
  'Dicom fpath': 'C:\\Temp\\2020-03-04\\From DICOM\\1.3.12.2.1107.5.2.18.41171.30000017030208045014800000004-3-1-1y5alqg.dcm',
  'Image pos': [-255.81113815308, -279.05569076538, 142.46780395508],
  'Image orient': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
  'Slice loc': 142.46780395508,
  'Pix spacing': [1.5625, 1.5625],
  'Pix array': array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]], dtype=uint16),
  'Closest Dicom SOP UID': '1.3.12.2.1107.5.2.18.41171.2017030211443692233924053',
  'Closest Dicom frame no': 1,
  'Closest Dicom fpath': 'C:\\Temp\\2020-03-04\\To DICOM\\1.3.12.2.1107.5.2.18.41171.30000017030208045014800000004-4-1-1i5qe5f.dcm',
  'Closest Image pos': [-255.81113815308, -168.55297851563, 149.91796875],
  'Closest Image orient': [1.0, 0.0, 0

In [173]:
roiDict1
dicomDict1
xnatDict

{'xnatAddress': 'http://10.1.1.17',
 'username': 'admin',
 'password': 'admin',
 'fromRoiProjLab': 'TestData',
 'fromRoiSubjLab': 'Volunteer_1',
 'fromRoiExpLab': '20170302_112746_Aera',
 'fromRoiLab': 'AIM_20200304_110009',
 'fromDicomProjLab': 'TestData',
 'fromDicomSubjLab': 'Volunteer_1',
 'fromDicomExpLab': '20170302_112746_Aera',
 'fromDicomScanLab': '3',
 'toDicomProjLab': 'TestData',
 'toDicomSubjLab': 'Volunteer_1',
 'toDicomExpLab': '20170302_112746_Aera',
 'toDicomScanLab': '4',
 'rootDir': 'C:\\Temp\\2020-03-04',
 'fromDicomDir': 'C:\\Temp\\2020-03-04\\From DICOM',
 'fromRoiDir': 'C:\\Temp\\2020-03-04\\From RTSTRUCT',
 'toDicomDir': 'C:\\Temp\\2020-03-04\\To DICOM',
 'toRoiDir': 'C:\\Temp\\2020-03-04\\To RTSTRUCT'}

# Try a different approach to ModifyROIs function:

In [157]:
import DicomHelperFuncs
importlib.reload(DicomHelperFuncs)
from DicomHelperFuncs import GetDicomData
from DicomHelperFuncs import GetRoiData

# Inputs:
#### dicomAndRoiDict1 
#### dicomDict2
# roiDict1
# dicomDict1_to_2


# Outputs:
#### dicomAndRoiDict2
#### roi2
# roiDict2

# Initialise dicomAndRoiDict2 by starting with a copy of dicomDict2:
#dicomAndRoiDict2 = copy.deepcopy(dicomDict2)

# Copy roi1:
roi2 = copy.deepcopy(roi1)

# Get the keys in dicomAndRoiDict1, dicomDict2 and roi1:
#Dkeys1 = list(dicomAndRoiDict1.keys())
D1_to_2_keys = list(dicomDict1_to_2.keys())
D2_keys = list(dicomDict2.keys())
R1_keys = list(roiDict1.keys())



# Cycle through each key (Ref SOP UID) in roiDict1:
for R1_key in R1_keys:
    # Find the index of the key (SOP UID) in dicomDict1_to_2 that matches
    # the key (Ref SOP UID) in roiDict1:
    ind_D1_to_R1 = D1_to_2_keys.index(R1_key)
    
    # The SOP UID in dicomDict2 that is closest in postion to the slice 
    # that the contour came from:
    #uid = dicomDict1_to_2[ind_D1_to_R1]['Closest Dicom SOP UID']
    
    # The filepath in dicomDict2 that is closest in postion to the slice 
    # that the contour came from:
    fpath2 = dicomDict1_to_2[ind_D1_to_R1]['Closest Dicom fpath']
    
    # Load the DICOM from the new series:
    dicom2 = pydicom.read_file(fpath2)
    
    
    

{'1.3.12.2.1107.5.2.18.41171.2017030211443692233924053': {'Dicom frame no': 1,
  'Dicom fpath': 'C:\\Temp\\2020-03-04\\To DICOM\\1.3.12.2.1107.5.2.18.41171.30000017030208045014800000004-4-1-1i5qe5f.dcm',
  'Image pos': [-255.81113815308, -168.55297851563, 149.91796875],
  'Image orient': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
  'Slice loc': 149.91796875,
  'Pix spacing': [1.5625, 1.5625],
  'Pix array': array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]], dtype=uint16),
  'Closest Dicom frame no': 1,
  'Closest Dicom fpath': 'C:\\Temp\\2020-03-04\\From DICOM\\1.3.12.2.1107.5.2.18.41171.30000017030208045014800000004-3-1-1y5alqg.dcm',
  'Closest Image pos': [-255.81113815308, -279.05569076538, 142.46780395508],
  'Closest Image orient': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
  'Closest Slice loc': 142.46780395508,
  'Closest Pix spacing': [

In [177]:
xnatDict

{'xnatAddress': 'http://10.1.1.17',
 'username': 'admin',
 'password': 'admin',
 'fromRoiProjLab': 'TestData',
 'fromRoiSubjLab': 'Volunteer_1',
 'fromRoiExpLab': '20170302_112746_Aera',
 'fromRoiLab': 'AIM_20200304_110009',
 'fromDicomProjLab': 'TestData',
 'fromDicomSubjLab': 'Volunteer_1',
 'fromDicomExpLab': '20170302_112746_Aera',
 'fromDicomScanLab': '3',
 'toDicomProjLab': 'TestData',
 'toDicomSubjLab': 'Volunteer_1',
 'toDicomExpLab': '20170302_112746_Aera',
 'toDicomScanLab': '4',
 'rootDir': 'C:\\Temp\\2020-03-04',
 'fromDicomDir': 'C:\\Temp\\2020-03-04\\From DICOM',
 'fromRoiDir': 'C:\\Temp\\2020-03-04\\From RTSTRUCT',
 'toDicomDir': 'C:\\Temp\\2020-03-04\\To DICOM',
 'toRoiDir': 'C:\\Temp\\2020-03-04\\To RTSTRUCT'}

In [178]:
roiDict1

{'1.3.12.2.1107.5.2.18.41171.2017030211424716719523153': {'Roi fpath': 'C:\\Temp\\2020-03-04\\From RTSTRUCT\\AIM_20200304_110009.dcm',
  'No of contours': 3,
  'Contour nos': [1, 2, 3],
  'No of contour pts': [44, 45, 116],
  'Contour data': [[-18.083705824172,
    -62.778718089179,
    132.86779785156,
    -18.594431053999,
    -61.2465423997,
    132.86779785156,
    -19.105156283825,
    -59.714366710221,
    132.86779785156,
    -19.105156283825,
    -57.671465790916,
    132.86779785156,
    -17.06225536452,
    -57.671465790916,
    132.86779785156,
    -15.019354445214,
    -57.671465790916,
    132.86779785156,
    -13.487178755735,
    -56.650015331263,
    132.86779785156,
    -11.955003066256,
    -56.139290101437,
    132.86779785156,
    -10.422827376777,
    -55.117839641784,
    132.86779785156,
    -9.9121021469513,
    -53.585663952305,
    132.86779785156,
    -9.401376917125,
    -52.053488262826,
    132.86779785156,
    -7.8692012276459,
    -51.542763033,
    132.

# Test out the functions that were created based on the code above:

In [4]:
xnatAddress = 'http://10.1.1.17'

session = xnat.connect(xnatAddress, user='admin', password='admin')
session

In [231]:
# Create a dictionary to store various variables to input into CopyROI():

todaysDate = time.strftime("%Y-%m-%d", time.gmtime())

rootDir = os.path.join(r'C:\Temp', todaysDate)

xnatDict = {# Define the XNAT login details:
            'xnatAddress':'http://10.1.1.17',\
            'username':'admin',\
            'password':'admin',\
    
            # Define the REST variables that point to the ROIs to be copied:
            'fromRoiProjLab':'TestData',\
            'fromRoiSubjLab':'Volunteer_1',\
            'fromRoiExpLab':'20170302_112746_Aera',\
            'fromRoiLab':'AIM_20200304_110009',\
    
            # Define the REST variables that point to the DICOM files that the ROI applies to:
            'fromDicomProjLab':'TestData',\
            'fromDicomSubjLab':'Volunteer_1',\
            'fromDicomExpLab':'20170302_112746_Aera',\
            'fromDicomScanLab':'3',\
    
            # Define the REST variables that point to the DICOM files that the ROI is to be applied to:
            'toDicomProjLab':'TestData',\
            'toDicomSubjLab':'Volunteer_1',\
            'toDicomExpLab':'20170302_112746_Aera',\
            'toDicomScanLab':'4',\
    
            # Define the label to assign to Structure Set Label and to ROI Name of the new RTSTUCT file:
            #'toRoiStructSetLab' = 'ROIs_t2_blade_tra_slice_5'
    
    
            # Define the directories to download files to:
            'rootDir':rootDir, \
            'fromDicomDir':os.path.join(rootDir, 'From DICOM'),\
            'fromRoiDir':os.path.join(rootDir, 'From RTSTRUCT'),\
            'toDicomDir':os.path.join(rootDir, 'To DICOM'),\
            'toRoiDir':os.path.join(rootDir, 'To RTSTRUCT')
           }

xnatDict

{'xnatAddress': 'http://10.1.1.17',
 'username': 'admin',
 'password': 'admin',
 'fromRoiProjLab': 'TestData',
 'fromRoiSubjLab': 'Volunteer_1',
 'fromRoiExpLab': '20170302_112746_Aera',
 'fromRoiLab': 'AIM_20200304_110009',
 'fromDicomProjLab': 'TestData',
 'fromDicomSubjLab': 'Volunteer_1',
 'fromDicomExpLab': '20170302_112746_Aera',
 'fromDicomScanLab': '3',
 'toDicomProjLab': 'TestData',
 'toDicomSubjLab': 'Volunteer_1',
 'toDicomExpLab': '20170302_112746_Aera',
 'toDicomScanLab': '4',
 'rootDir': 'C:\\Temp\\2020-03-06',
 'fromDicomDir': 'C:\\Temp\\2020-03-06\\From DICOM',
 'fromRoiDir': 'C:\\Temp\\2020-03-06\\From RTSTRUCT',
 'toDicomDir': 'C:\\Temp\\2020-03-06\\To DICOM',
 'toRoiDir': 'C:\\Temp\\2020-03-06\\To RTSTRUCT'}

In [232]:
import DicomHelperFuncs
importlib.reload(DicomHelperFuncs)
from DicomHelperFuncs import DownloadFilesForRoiCopy
from DicomHelperFuncs import GetDicomData
from DicomHelperFuncs import GetRoiData
from DicomHelperFuncs import GetClosestDicomPositions
from DicomHelperFuncs import ModifyRoi

# Download the DICOM-RTSTRUCT and DICOM files from XNAT:
xnatDict = DownloadFilesForRoiCopy(xnatDict, del_dicoms=False, debug=False)

100% of  12.2 KiB |################################|  11.9 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  91.4 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################| 121.9 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  91.5 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  91.5 MiB/s Time:  0:00:00


100% of 374.6 KiB |################################| 121.9 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  91.5 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################| 121.9 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  91.4 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################| 121.9 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################| 121.9 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  73.1 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################| 121.9 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################| 121.9 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  60.9 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  91.5 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################| 121.9 MiB/s Time:  0:00:00
100% of 374.6 KiB |####################

100% of 374.6 KiB |################################| 121.8 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  92.2 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  73.6 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  91.5 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  91.4 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################| 121.6 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################| 104.4 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  73.4 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################| 122.1 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################| 121.9 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  91.4 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################| 122.0 MiB/s Time:  0:00:00
100% of 374.6 KiB |#####################

100% of 293.7 KiB |################################|  95.6 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################| 143.5 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################| 143.6 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  95.6 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  82.0 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  95.5 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  95.6 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  71.6 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  95.8 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  85.1 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################| 190.2 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  71.4 MiB/s Time:  0:00:00
100% of 293.7 KiB |#####################

100% of 293.7 KiB |################################| 143.4 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  71.7 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  95.6 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  71.7 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  57.4 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  57.4 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################| 143.5 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  71.8 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  71.7 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################| 143.3 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################| 143.3 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################| 143.5 MiB/s Time:  0:00:00
100% of 293.7 KiB |#####################

In [233]:
# Get the ROI data to be copied:
roiDict1 = GetRoiData(xnatDict['fromRoiFpath'])

roiDict1

{'1.3.12.2.1107.5.2.18.41171.2017030211424716719523153': {'Roi fpath': 'C:\\Temp\\2020-03-06\\From RTSTRUCT\\AIM_20200304_110009.dcm',
  'No of contours': 3,
  'Contour nos': [1, 2, 3],
  'No of contour pts': [44, 45, 116],
  'Contour data': [[-18.083705824172,
    -62.778718089179,
    132.86779785156,
    -18.594431053999,
    -61.2465423997,
    132.86779785156,
    -19.105156283825,
    -59.714366710221,
    132.86779785156,
    -19.105156283825,
    -57.671465790916,
    132.86779785156,
    -17.06225536452,
    -57.671465790916,
    132.86779785156,
    -15.019354445214,
    -57.671465790916,
    132.86779785156,
    -13.487178755735,
    -56.650015331263,
    132.86779785156,
    -11.955003066256,
    -56.139290101437,
    132.86779785156,
    -10.422827376777,
    -55.117839641784,
    132.86779785156,
    -9.9121021469513,
    -53.585663952305,
    132.86779785156,
    -9.401376917125,
    -52.053488262826,
    132.86779785156,
    -7.8692012276459,
    -51.542763033,
    132.

In [234]:
# Get the DICOM data that corresponds to the ROI data to be copied:
dicomDict1 = GetDicomData(xnatDict['fromDicomDir'])

dicomDict1

{'1.3.12.2.1107.5.2.18.41171.2017030211424716885123155': {'Dicom frame no': 1,
  'Dicom fpath': 'C:\\Temp\\2020-03-06\\From DICOM\\1.3.12.2.1107.5.2.18.41171.30000017030208045014800000004-3-1-1y5alqg.dcm',
  'Image pos': [-255.81113815308, -279.05569076538, 142.46780395508],
  'Image orient': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
  'Slice loc': 142.46780395508,
  'Pix spacing': [1.5625, 1.5625],
  'Pix array': array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]], dtype=uint16)},
 '1.3.12.2.1107.5.2.18.41171.2017030211424737916723219': {'Dicom frame no': 2,
  'Dicom fpath': 'C:\\Temp\\2020-03-06\\From DICOM\\1.3.12.2.1107.5.2.18.41171.30000017030208045014800000004-3-2-6f3ot1.dcm',
  'Image pos': [-255.81113815308, -279.05569076538, 140.0677947998],
  'Image orient': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
  'Slice loc': 140.0677947998,
  'P

In [235]:
# Get the DICOM data of the scan that ROI data is to be copied to:
dicomDict2 = GetDicomData(xnatDict['toDicomDir'])

dicomDict2

{'1.3.12.2.1107.5.2.18.41171.2017030211443692233924053': {'Dicom frame no': 1,
  'Dicom fpath': 'C:\\Temp\\2020-03-06\\To DICOM\\1.3.12.2.1107.5.2.18.41171.30000017030208045014800000004-4-1-1i5qe5f.dcm',
  'Image pos': [-255.81113815308, -168.55297851563, 149.91796875],
  'Image orient': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
  'Slice loc': 149.91796875,
  'Pix spacing': [1.5625, 1.5625],
  'Pix array': array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]], dtype=uint16)},
 '1.3.12.2.1107.5.2.18.41171.201703021144378761324117': {'Dicom frame no': 2,
  'Dicom fpath': 'C:\\Temp\\2020-03-06\\To DICOM\\1.3.12.2.1107.5.2.18.41171.30000017030208045014800000004-4-2-oocxno.dcm',
  'Image pos': [-255.81113815308, -168.55297851563, 147.51797485352],
  'Image orient': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
  'Slice loc': 147.51797485352,
  'Pix spacin

In [236]:
import DicomHelperFuncs
importlib.reload(DicomHelperFuncs)
from DicomHelperFuncs import GetClosestDicomPositions

# Get the mapping from the 1st to 2nd DICOM scans using the z-coordinate 
# to find the nearest slice:
dicomDict1_to_2 = GetClosestDicomPositions(dicomDict1, dicomDict2, searchBy='z')

dicomDict1_to_2

{'1.3.12.2.1107.5.2.18.41171.2017030211424716885123155': {'Dicom frame no': 1,
  'Dicom fpath': 'C:\\Temp\\2020-03-06\\From DICOM\\1.3.12.2.1107.5.2.18.41171.30000017030208045014800000004-3-1-1y5alqg.dcm',
  'Image pos': [-255.81113815308, -279.05569076538, 142.46780395508],
  'Image orient': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
  'Slice loc': 142.46780395508,
  'Pix spacing': [1.5625, 1.5625],
  'Pix array': array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]], dtype=uint16),
  'Closest Dicom SOP UID': '1.3.12.2.1107.5.2.18.41171.201703021144378736224116',
  'Closest Dicom frame no': 4,
  'Closest Dicom fpath': 'C:\\Temp\\2020-03-06\\To DICOM\\1.3.12.2.1107.5.2.18.41171.30000017030208045014800000004-4-4-b4fb70.dcm',
  'Closest Image pos': [-255.81113815308, -168.55297851563, 142.71797180176],
  'Closest Image orient': [1.0, 0.0, 

In [229]:
import DicomHelperFuncs
importlib.reload(DicomHelperFuncs)
from DicomHelperFuncs import IndexDictInTier2

key1 = IndexDictInTier2(dicomDict1, tier2key='Dicom frame no', tier2val=5)


print('Contour copied from Image Position =', dicomDict1[key1]['Image pos'], '\n')

# loop through each key in dicomDict2:
keys2 = list(dicomDict2.keys())

for i in range(len(keys2)):
    print(f'Slice {i+1} Image Position =', dicomDict2[keys2[i]]['Image pos'])

Contour copied from Image Position = [-255.81113815308, -279.05569076538, 132.86779785156] 

Slice 1 Image Position = [-255.81113815308, -168.55297851563, 149.91796875]
Slice 2 Image Position = [-255.81113815308, -168.55297851563, 147.51797485352]
Slice 3 Image Position = [-255.81113815308, -168.55297851563, 145.11798095703]
Slice 4 Image Position = [-255.81113815308, -168.55297851563, 142.71797180176]
Slice 5 Image Position = [-255.81113815308, -168.55297851563, 140.31797790527]
Slice 6 Image Position = [-255.81113815308, -168.55297851563, 137.91796875]
Slice 7 Image Position = [-255.81113815308, -168.55297851563, 135.51797485352]
Slice 8 Image Position = [-255.81113815308, -168.55297851563, 133.11798095703]
Slice 9 Image Position = [-255.81113815308, -168.55297851563, 130.71797180176]
Slice 10 Image Position = [-255.81113815308, -168.55297851563, 128.31797790527]
Slice 11 Image Position = [-255.81113815308, -168.55297851563, 125.91797637939]
Slice 12 Image Position = [-255.8111381530

In [240]:
import DicomHelperFuncs
importlib.reload(DicomHelperFuncs)
from DicomHelperFuncs import ModifyRoi

# Copy and modify the ROIs from the 1st DICOM scan to overlay on the 2nd DICOM scan:
roi2, xnatDict = ModifyRoi(xnatDict, dicomDict1_to_2)

xnatDict


The Study Date are the same:
 20170302

Changing the Study Time from:
 114247 
to:
 112746

The Patient Name are the same:
 Volunteer_1

The Patient ID are the same:
 20170302_112746_Aera

Patient Birth Date does not exist in the DICOMs

The Patient Sex are the same:
 F

The Study Instance UID are the same:
 1.3.12.2.1107.5.2.18.41171.30000017030208045014800000004

The Study Instance UID are the same:
 1.3.12.2.1107.5.2.18.41171.30000017030208045014800000004

Changing the Series Instance UID from:
 1.3.12.2.1107.5.2.18.41171.201703021136315076922448.0.0.0 
to:
 1.3.12.2.1107.5.2.18.41171.2017030211424753515923220.0.0.0

The Frame Of ReferenceUID UID are the same:
 1.3.12.2.1107.5.2.18.41171.2.20170302112746520.0.0.0

The Frame Of ReferenceUID UID are the same:
 1.3.12.2.1107.5.2.18.41171.2.20170302112746520.0.0.0

The Referenced Frame Of ReferenceUID UID is the same as the Frame Of Reference UID:
 1.3.12.2.1107.5.2.18.41171.2.20170302112746520.0.0.0

Changing the Structure Set Label f

{'xnatAddress': 'http://10.1.1.17',
 'username': 'admin',
 'password': 'admin',
 'fromRoiProjLab': 'TestData',
 'fromRoiSubjLab': 'Volunteer_1',
 'fromRoiExpLab': '20170302_112746_Aera',
 'fromRoiLab': 'AIM_20200304_110009',
 'fromDicomProjLab': 'TestData',
 'fromDicomSubjLab': 'Volunteer_1',
 'fromDicomExpLab': '20170302_112746_Aera',
 'fromDicomScanLab': '3',
 'toDicomProjLab': 'TestData',
 'toDicomSubjLab': 'Volunteer_1',
 'toDicomExpLab': '20170302_112746_Aera',
 'toDicomScanLab': '4',
 'rootDir': 'C:\\Temp\\2020-03-06',
 'fromDicomDir': 'C:\\Temp\\2020-03-06\\From DICOM',
 'fromRoiDir': 'C:\\Temp\\2020-03-06\\From RTSTRUCT',
 'toDicomDir': 'C:\\Temp\\2020-03-06\\To DICOM',
 'toRoiDir': 'C:\\Temp\\2020-03-06\\To RTSTRUCT',
 'fromRoiFname': 'AIM_20200304_110009.dcm',
 'fromRoiFpath': 'C:\\Temp\\2020-03-06\\From RTSTRUCT\\AIM_20200304_110009.dcm',
 'changesToRoi': {'fromStudyTime': '114247',
  'toStudyTime': '112746',
  'fromRFORS.RTRSS.RTRSS.SeriesInstanceUID': '1.3.12.2.1107.5.2.18

In [238]:
roi2

(0008, 0016) SOP Class UID                       UI: RT Structure Set Storage
(0008, 0018) SOP Instance UID                    UI: 1.2.826.0.1.534147.578.2126823365.20202413735791
(0008, 0020) Study Date                          DA: '20170302'
(0008, 0030) Study Time                          TM: '112746'
(0008, 0050) Accession Number                    SH: ''
(0008, 0060) Modality                            CS: 'RTSTRUCT'
(0008, 0070) Manufacturer                        LO: 'SIEMENS'
(0008, 0090) Referring Physician's Name          PN: ''
(0008, 1070) Operators' Name                     PN: ''
(0008, 1090) Manufacturer's Model Name           LO: 'Aera'
(0010, 0010) Patient's Name                      PN: 'Volunteer_1'
(0010, 0020) Patient ID                          LO: '20170302_112746_Aera'
(0010, 0030) Patient's Birth Date                DA: '00021130'
(0010, 0040) Patient's Sex                       CS: 'F'
(0018, 1020) Software Version(s)                 LO: 'syngo MR E11'
(0020, 

In [194]:
dicomDict1_to_2

{'1.3.12.2.1107.5.2.18.41171.2017030211424716885123155': {'Dicom frame no': 1,
  'Dicom fpath': 'C:\\Temp\\2020-03-05\\From DICOM\\1.3.12.2.1107.5.2.18.41171.30000017030208045014800000004-3-1-1y5alqg.dcm',
  'Image pos': [-255.81113815308, -279.05569076538, 142.46780395508],
  'Image orient': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
  'Slice loc': 142.46780395508,
  'Pix spacing': [1.5625, 1.5625],
  'Pix array': array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]], dtype=uint16),
  'Closest Dicom SOP UID': '1.3.12.2.1107.5.2.18.41171.2017030211443692233924053',
  'Closest Dicom frame no': 1,
  'Closest Dicom fpath': 'C:\\Temp\\2020-03-05\\To DICOM\\1.3.12.2.1107.5.2.18.41171.30000017030208045014800000004-4-1-1i5qe5f.dcm',
  'Closest Image pos': [-255.81113815308, -168.55297851563, 149.91796875],
  'Closest Image orient': [1.0, 0.0, 0

In [241]:
xnatDict

{'xnatAddress': 'http://10.1.1.17',
 'username': 'admin',
 'password': 'admin',
 'fromRoiProjLab': 'TestData',
 'fromRoiSubjLab': 'Volunteer_1',
 'fromRoiExpLab': '20170302_112746_Aera',
 'fromRoiLab': 'AIM_20200304_110009',
 'fromDicomProjLab': 'TestData',
 'fromDicomSubjLab': 'Volunteer_1',
 'fromDicomExpLab': '20170302_112746_Aera',
 'fromDicomScanLab': '3',
 'toDicomProjLab': 'TestData',
 'toDicomSubjLab': 'Volunteer_1',
 'toDicomExpLab': '20170302_112746_Aera',
 'toDicomScanLab': '4',
 'rootDir': 'C:\\Temp\\2020-03-06',
 'fromDicomDir': 'C:\\Temp\\2020-03-06\\From DICOM',
 'fromRoiDir': 'C:\\Temp\\2020-03-06\\From RTSTRUCT',
 'toDicomDir': 'C:\\Temp\\2020-03-06\\To DICOM',
 'toRoiDir': 'C:\\Temp\\2020-03-06\\To RTSTRUCT',
 'fromRoiFname': 'AIM_20200304_110009.dcm',
 'fromRoiFpath': 'C:\\Temp\\2020-03-06\\From RTSTRUCT\\AIM_20200304_110009.dcm',
 'changesToRoi': {'fromStudyTime': '114247',
  'toStudyTime': '112746',
  'fromRFORS.RTRSS.RTRSS.SeriesInstanceUID': '1.3.12.2.1107.5.2.18

## After finding the bug in the function GetClosestDicomPositions() I finally ended up with the contours from slice 5 in the original set copied to slice 8 in the new set, overlapping as expected despite the change in image resolution.

### Although the above worked see if I get the same results after wrapping everything up in a function:

In [287]:
import DicomHelperFuncs
importlib.reload(DicomHelperFuncs)
from DicomHelperFuncs import CopyRoi

# Run CopyROI() to do everything:
roi2, xnatDict = CopyRoi(fromRoiProjLab='TestData', \
                         fromRoiSubjLab='Volunteer_1', \
                         fromRoiExpLab='20170302_112746_Aera', \
                         fromRoiLabDateTime='20200304_110009', \
                         fromDicomScanLab='3', \
                         toDicomProjLab='TestData', \
                         toDicomSubjLab='Volunteer_1', \
                         toDicomExpLab='20170302_112746_Aera', \
                         toDicomScanLab='4', \
                         searchBy='z', \
                         del_dicoms=True, \
                         debug=False)

xnatDict

100% of  12.2 KiB |################################|   5.8 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################| 121.7 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  45.7 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  91.5 MiB/s Time:  0:00:00


100% of 374.6 KiB |################################|  91.5 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  73.2 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  61.0 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  45.7 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  91.4 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  52.3 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  45.7 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  40.6 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  73.1 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  73.1 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  91.5 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  91.5 MiB/s Time:  0:00:00
100% of 374.6 KiB |#####################

100% of 374.6 KiB |################################|  61.0 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  40.7 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  52.3 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  61.0 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  22.8 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  61.0 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  91.4 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  73.1 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  73.2 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  60.9 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  45.7 MiB/s Time:  0:00:00
100% of 374.6 KiB |################################|  40.7 MiB/s Time:  0:00:00
100% of 374.6 KiB |#####################

100% of 293.7 KiB |################################|  71.7 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  95.3 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  71.7 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  94.7 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  71.6 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################| 143.2 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  71.8 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  95.5 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  71.7 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  71.7 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  57.3 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  57.4 MiB/s Time:  0:00:00
100% of 293.7 KiB |#####################

100% of 293.7 KiB |################################|  41.0 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  11.5 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  47.8 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  26.1 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  95.6 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  41.0 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  31.9 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  95.6 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  47.8 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  16.9 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  57.4 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  95.4 MiB/s Time:  0:00:00
100% of 293.7 KiB |#####################


Took 12.9 s to download data from  XNAT

The Study Date are the same:
 20170302

Changing the Study Time from:
 114247 
to:
 112746

The Patient Name are the same:
 Volunteer_1

The Patient ID are the same:
 20170302_112746_Aera

Patient Birth Date does not exist in the DICOMs

The Patient Sex are the same:
 F

The Study Instance UID are the same:
 1.3.12.2.1107.5.2.18.41171.30000017030208045014800000004

The Study Instance UID are the same:
 1.3.12.2.1107.5.2.18.41171.30000017030208045014800000004

Changing the Series Instance UID from:
 1.3.12.2.1107.5.2.18.41171.201703021136315076922448.0.0.0 
to:
 1.3.12.2.1107.5.2.18.41171.2017030211424753515923220.0.0.0

The Frame Of ReferenceUID UID are the same:
 1.3.12.2.1107.5.2.18.41171.2.20170302112746520.0.0.0

The Frame Of ReferenceUID UID are the same:
 1.3.12.2.1107.5.2.18.41171.2.20170302112746520.0.0.0

The Referenced Frame Of ReferenceUID UID is the same as the Frame Of Reference UID:
 1.3.12.2.1107.5.2.18.41171.2.20170302112746520.

{'xnatAddress': 'http://10.1.1.17',
 'username': 'admin',
 'password': 'admin',
 'fromRoiProjLab': 'TestData',
 'fromRoiSubjLab': 'Volunteer_1',
 'fromRoiExpLab': '20170302_112746_Aera',
 'fromRoiLabDateTime': '20200304_110009',
 'fromDicomScanLab': '3',
 'toDicomProjLab': 'TestData',
 'toDicomSubjLab': 'Volunteer_1',
 'toDicomExpLab': '20170302_112746_Aera',
 'toDicomScanLab': '4',
 'rootDir': 'C:\\Temp\\2020-03-06',
 'fromDicomDir': 'C:\\Temp\\2020-03-06\\From DICOM',
 'fromRoiDir': 'C:\\Temp\\2020-03-06\\From RTSTRUCT',
 'toDicomDir': 'C:\\Temp\\2020-03-06\\To DICOM',
 'toRoiDir': 'C:\\Temp\\2020-03-06\\To RTSTRUCT',
 'fromRoiFname': 'AIM_20200304_110009.dcm',
 'fromRoiFpath': 'C:\\Temp\\2020-03-06\\From RTSTRUCT\\AIM_20200304_110009.dcm',
 'changesToRoi': {'fromStudyTime': '114247',
  'toStudyTime': '112746',
  'fromRFORS.RTRSS.RTRSS.SeriesInstanceUID': '1.3.12.2.1107.5.2.18.41171.201703021136315076922448.0.0.0',
  'toRFORS.RTRSS.RTRSS.SeriesInstanceUID': '1.3.12.2.1107.5.2.18.4117

### Seems fine.  Now onto the next exercise...

# Exercise 2 - Copy ROI Collection to scan with different resolution and different number of planes:

### Copy ROI Collection from scan 4 ("t2_tra", res = 320 x 190, 128 slices) to scan 7 ("Dixon_T1_opp", res = 384 x 240, 192 slices) 

In [288]:
import DicomHelperFuncs
importlib.reload(DicomHelperFuncs)
from DicomHelperFuncs import CopyRoi

# Run CopyROI() to do everything:
roi2, xnatDict = CopyRoi(fromRoiProjLab='TestData', \
                         fromRoiSubjLab='Volunteer_1', \
                         fromRoiExpLab='20170302_112746_Aera', \
                         fromRoiLabDateTime='20200306_114104', \
                         fromDicomScanLab='4', \
                         toDicomProjLab='TestData', \
                         toDicomSubjLab='Volunteer_1', \
                         toDicomExpLab='20170302_112746_Aera', \
                         toDicomScanLab='7', \
                         searchBy='z', \
                         del_dicoms=True, \
                         debug=False)

xnatDict

100% of  12.3 KiB |################################|  12.0 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  71.6 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  71.7 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  57.4 MiB/s Time:  0:00:00


100% of 293.7 KiB |################################|  47.8 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  57.4 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  57.4 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  47.8 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  57.4 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  47.7 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  41.0 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  57.4 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  47.8 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  47.8 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  28.7 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  35.9 MiB/s Time:  0:00:00
100% of 293.7 KiB |#####################

100% of 293.7 KiB |################################|  57.3 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  57.4 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  57.4 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  95.7 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  71.7 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  57.4 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  47.8 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  56.7 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  52.3 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  62.3 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  71.7 MiB/s Time:  0:00:00
100% of 293.7 KiB |################################|  71.7 MiB/s Time:  0:00:00
100% of 293.7 KiB |#####################

100% of 292.2 KiB |################################|  69.3 MiB/s Time:  0:00:00
100% of 292.2 KiB |################################|  71.3 MiB/s Time:  0:00:00
100% of 292.2 KiB |################################|  71.3 MiB/s Time:  0:00:00
100% of 292.2 KiB |################################|  71.3 MiB/s Time:  0:00:00
100% of 292.2 KiB |################################|  57.1 MiB/s Time:  0:00:00
100% of 292.2 KiB |################################|  71.3 MiB/s Time:  0:00:00
100% of 292.2 KiB |################################|  95.1 MiB/s Time:  0:00:00
100% of 292.2 KiB |################################|  57.1 MiB/s Time:  0:00:00
100% of 292.2 KiB |################################|  25.9 MiB/s Time:  0:00:00
100% of 292.2 KiB |################################|  57.1 MiB/s Time:  0:00:00
100% of 292.2 KiB |################################|  95.1 MiB/s Time:  0:00:00
100% of 292.2 KiB |################################|  95.1 MiB/s Time:  0:00:00
100% of 292.2 KiB |#####################

100% of 292.2 KiB |################################|  56.7 MiB/s Time:  0:00:00
100% of 292.2 KiB |################################|  71.4 MiB/s Time:  0:00:00
100% of 292.2 KiB |################################|  71.9 MiB/s Time:  0:00:00
100% of 292.2 KiB |################################|  95.0 MiB/s Time:  0:00:00
100% of 292.2 KiB |################################|  71.7 MiB/s Time:  0:00:00
100% of 292.2 KiB |################################|  57.1 MiB/s Time:  0:00:00
100% of 292.2 KiB |################################|  95.2 MiB/s Time:  0:00:00
100% of 292.2 KiB |################################|  35.7 MiB/s Time:  0:00:00
100% of 292.2 KiB |################################|  31.7 MiB/s Time:  0:00:00
100% of 292.2 KiB |################################|  57.1 MiB/s Time:  0:00:00
100% of 292.2 KiB |################################|  95.2 MiB/s Time:  0:00:00
100% of 292.2 KiB |################################|  71.3 MiB/s Time:  0:00:00
100% of 292.2 KiB |#####################


Took 15.2 s to download data from  XNAT

The Study Date are the same:
 20170302

The Study Time are the same:
 112746

The Patient Name are the same:
 Volunteer_1

The Patient ID are the same:
 20170302_112746_Aera

Patient Birth Date does not exist in the DICOMs

The Patient Sex are the same:
 F

The Study Instance UID are the same:
 1.3.12.2.1107.5.2.18.41171.30000017030208045014800000004

The Study Instance UID are the same:
 1.3.12.2.1107.5.2.18.41171.30000017030208045014800000004

Changing the Series Instance UID from:
 1.3.12.2.1107.5.2.18.41171.2017030211424753515923220.0.0.0 
to:
 1.3.12.2.1107.5.2.18.41171.2017030212054840754728067.0.0.0

The Frame Of ReferenceUID UID are the same:
 1.3.12.2.1107.5.2.18.41171.2.20170302112746520.0.0.0

The Frame Of ReferenceUID UID are the same:
 1.3.12.2.1107.5.2.18.41171.2.20170302112746520.0.0.0

The Referenced Frame Of ReferenceUID UID is the same as the Frame Of Reference UID:
 1.3.12.2.1107.5.2.18.41171.2.20170302112746520.0.0.0

Changi

{'xnatAddress': 'http://10.1.1.17',
 'username': 'admin',
 'password': 'admin',
 'fromRoiProjLab': 'TestData',
 'fromRoiSubjLab': 'Volunteer_1',
 'fromRoiExpLab': '20170302_112746_Aera',
 'fromRoiLabDateTime': '20200306_114104',
 'fromDicomScanLab': '4',
 'toDicomProjLab': 'TestData',
 'toDicomSubjLab': 'Volunteer_1',
 'toDicomExpLab': '20170302_112746_Aera',
 'toDicomScanLab': '7',
 'rootDir': 'C:\\Temp\\2020-03-06',
 'fromDicomDir': 'C:\\Temp\\2020-03-06\\From DICOM',
 'fromRoiDir': 'C:\\Temp\\2020-03-06\\From RTSTRUCT',
 'toDicomDir': 'C:\\Temp\\2020-03-06\\To DICOM',
 'toRoiDir': 'C:\\Temp\\2020-03-06\\To RTSTRUCT',
 'fromRoiFname': 'RTSTRUCT_20200306_114104.dcm',
 'fromRoiFpath': 'C:\\Temp\\2020-03-06\\From RTSTRUCT\\RTSTRUCT_20200306_114104.dcm',
 'changesToRoi': {'fromRFORS.RTRSS.RTRSS.SeriesInstanceUID': '1.3.12.2.1107.5.2.18.41171.2017030211424753515923220.0.0.0',
  'toRFORS.RTRSS.RTRSS.SeriesInstanceUID': '1.3.12.2.1107.5.2.18.41171.2017030212054840754728067.0.0.0',
  'fromSt